In [1]:
import pandas as pd
import pyodbc
import logging
import requests
import copy
import math
import time
import nltk
from elasticsearch import helpers

from elasticsearch import Elasticsearch


In [ ]:
def get_connection(db_config_path):
    with open(db_config_path, encoding='utf8') as db:
        database = ';'.join(db.readlines()[1:]).replace('\n', '').replace(' ', '') + ';'
    return pyodbc.connect('DRIVER={/opt/microsoft/msodbcsql18/lib64/libmsodbcsql-18.3.so.2.1};' + database + 'TrustServerCertificate=yes;')

In [ ]:
con = get_connection('materiaisServicos/config.ini')
sqlMaterial = """SELECT 
            m.ID_MATERIAL, 
            p.TIPO, 
            M.CODIGO_MATERIAL, 
            M.ITEM_SUSPENSO,
            M.MATERIAL_SUSTENTAVEL,
            c.CODIGO_CLASSE,
            c.DESCRICAO_CLASSE,
            s.CODIGO_SUBCLASSE,
            s.DESCRICAO_SUBCLASSE,
            g.CODIGO_GRUPO,
            g.DESCRICAO_GRUPO,
            d.CODIGO_DIVISAO,
            d.DESCRICAO_DIVISAO,
            s2.CODIGO_SECAO,
            s2.DESCRICAO_SECAO,
            ISNULL(STUFF((SELECT ' ' + CONCAT(c.CARACTERISTICA, ' ', CV.CARACTERISTICA_VALOR, cv.SIGLA_UNIDADE_MEDIDA)
                        FROM CATALOGO.MATERIAL_CARACTERISTICA mc
                        JOIN CATALOGO.PDM_CARACTERISTICA pc ON pc.ID_PDM_CARACTERISTICA = mc.ID_PDM_CARACTERISTICA
                        JOIN CATALOGO.CARACTERISTICAS c ON pc.ID_CARACTERISTICA = c.ID_CARACTERISTICA
                        JOIN CATALOGO.CARACTERISTICAS_VALORES cv ON cv.ID_CARACTERISTICA_VALOR = pc.ID_CARACTERISTICA_VALOR
                        WHERE mc.ID_MATERIAL = m.ID_MATERIAL
                        FOR XML PATH('')), 1, 1, ''), '') AS CARACTERISTICAS_BUSCA,
            p.NOME_MATERIAL, 
            CONCAT(p.NOME_MATERIAL, ' ', ISNULL(STUFF((SELECT ' ' + CONCAT(c.CARACTERISTICA, ' ', CV.CARACTERISTICA_VALOR, cv.SIGLA_UNIDADE_MEDIDA)
                                                    FROM CATALOGO.MATERIAL_CARACTERISTICA mc
                                                    JOIN CATALOGO.PDM_CARACTERISTICA pc ON pc.ID_PDM_CARACTERISTICA = mc.ID_PDM_CARACTERISTICA
                                                    JOIN CATALOGO.CARACTERISTICAS c ON pc.ID_CARACTERISTICA = c.ID_CARACTERISTICA
                                                    JOIN CATALOGO.CARACTERISTICAS_VALORES cv ON cv.ID_CARACTERISTICA_VALOR = pc.ID_CARACTERISTICA_VALOR
                                                    WHERE mc.ID_MATERIAL = m.ID_MATERIAL
                                                    FOR XML PATH('')), 1, 1, ''), '')) AS MATERIAL_TEXT
        FROM CATALOGO.MATERIAIS m
        JOIN CATALOGO.PDM p ON m.ID_PDM = p.ID_PDM
        LEFT JOIN CATALOGO.CLASSES c ON p.ID_CLASSE = c.ID_CLASSE
        LEFT JOIN CATALOGO.SUBCLASSES s ON c.ID_CLASSE = s.ID_CLASSE
        LEFT JOIN CATALOGO.GRUPOS g ON g.ID_GRUPO = c.ID_GRUPO
        LEFT JOIN CATALOGO.DIVISOES d ON d.ID_DIVISAO = g.ID_DIVISAO
        LEFT JOIN CATALOGO.SECOES s2 ON S2.ID_SECAO = D.ID_SECAO
        WHERE p.TIPO = 'M'
"""

In [ ]:
df_materiais = pd.read_sql_query(sqlMaterial, con)
df_materiais

In [ ]:
df_materiais = pd.read_csv('df_materiais.csv')

In [ ]:
classes_count = df_materiais['DESCRICAO_CLASSE'].value_counts()
classes_count = classes_count[classes_count < 100]
classes_count.index.tolist()


In [ ]:
classes_count = classes_count[classes_count > 5]
classes_count = classes_count.index.tolist()

In [ ]:
df_range_sample = df_materiais[df_materiais['DESCRICAO_CLASSE'].isin(classes_count)]
df_range_sample

In [ ]:
sample_catmat = df_range_sample.sample(50)
sample_catmat

In [ ]:
sample_catmat['MATERIAL_TEXT']

In [ ]:
sample_catmat = pd.read_csv('result_resample.csv')
sample_catmat

In [ ]:
itens_a_retirar = sample_catmat['ID_MATERIAL']
itens_a_retirar

In [ ]:
df_index_catmat = df_materiais[~df_materiais['ID_MATERIAL'].isin(itens_a_retirar)]
df_index_catmat

In [ ]:
sample_catmat = pd.read_csv('sample_catmat.csv')
sample_catmat


In [ ]:
df_index_catmat = pd.read_csv('df_index_catmat_resample.csv')
df_index_catmat

In [ ]:
descricoes_catmat = sample_catmat['MATERIAL_TEXT']
descricoes_catmat.to_csv('descricoes_sample_catmat.csv')

In [ ]:
sample_catmat['MATERIAL_TEXT']

# Dados para indexação

Retirando dados utilizados como teste de relevância do df a ser indexado

In [ ]:
df_index_catmat[df_index_catmat['MATERIAL_TEXT'].str.contains('Nome Agenda')][['ID_MATERIAL', 'MATERIAL_TEXT']]


In [ ]:
df_index_catmat.isnull().sum()

# RESAMPLE

In [ ]:
sample_catmat = pd.read_csv('result_resample.csv')
sample_catmat
new_sampl = df_materiais[df_materiais['ID_MATERIAL'].isin([581197, 508151])]
new_sampl

In [ ]:
sample_catmat = sample_catmat[~sample_catmat['ID_MATERIAL'].isin([508072, 581205])]
sample_catmat

In [ ]:
sample_catmat = pd.concat([new_sampl, sample_catmat], ignore_index=True)
sample_catmat

In [ ]:
sample_catmat['relevantes'] = None
for idx, query in sample_catmat.iterrows():
    nome_mater = query['NOME_MATERIAL']
    results_descricao = df_index_catmat[df_index_catmat['NOME_MATERIAL'] == nome_mater]
    sample_catmat['relevantes'][idx] = [(row['ID_MATERIAL'], 2) for _, row in results_descricao.iterrows()]
sample_catmat = sample_catmat[['ID_MATERIAL', 'NOME_MATERIAL', 'MATERIAL_TEXT', 'relevantes']]
sample_catmat
    

# Realizando indexação lexica



In [ ]:
es = Elasticsearch(
    "http://localhost:9202",
    basic_auth=('elastic', 'teste123'),
    request_timeout=999999999,
)

es.info()

In [ ]:
def doc_generator_lex(index_name, df):
  for idx, row in df.iterrows():
    yield {
        "_index": index_name,
        "_id": idx,
        "_source": {
           "id_material": row['ID_MATERIAL'],
            "codigo_classe": row['CODIGO_CLASSE'],
            "descricao_classe": row['DESCRICAO_CLASSE'],
            "codigo_grupo": row['CODIGO_GRUPO'],
            "caracteristica_busca": row['CARACTERISTICAS_BUSCA'],
            "nome_material": row['NOME_MATERIAL'],
            "material_text": row['MATERIAL_TEXT'],
        }
    }

In [ ]:
indices = es.cat.indices(h='index', format='json')
nomes_indices = [indice['index'] for indice in indices]
nomes_indices

In [ ]:
INDEX_NAME = "matlex"

In [ ]:
documentos_notas = doc_generator_lex(INDEX_NAME, df_index_catmat)

In [ ]:
es.options(ignore_status=[400,404]).indices.delete(index=INDEX_NAME)

body = {
   "settings": {
        'number_of_shards': 3,
        'number_of_replicas': 1,
        "analysis": {
            "analyzer": {
                "standard_asciifolding": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [ "lowercase", "asciifolding" ]
                }
            }
        }
    },
    
    'mappings': {
        'properties': {
            'id_material': {'type': 'long' },
            'codigo_classe': {'type': 'keyword' },
            'descricao_classe': {'type': 'text' },
            'codigo_grupo': {'type': 'keyword' },
            'caracteristica_busca': {'type': 'text' },
            'nome_material': {'type': 'text' },
            'material_text': {'type': 'text' },
        }
    }

}

es.indices.create(index=INDEX_NAME, body=body)

In [ ]:
indices = es.cat.indices(h='index', format='json')
nomes_indices = [indice['index'] for indice in indices]
nomes_indices

In [ ]:
try:
    resp = helpers.bulk(es, documentos_notas)

    if resp[1]:
        errors = resp[1]
        for error in errors:
            print("Falha de indexação:", error)
except helpers.BulkIndexError as e:
    for err in e.errors:
        print("Falha de indexação:", err)



Teste de query a indexação lexica
 

In [ ]:
query = {
    "query": {
        "match_all": {}
    },
}

resp = es.search(index=INDEX_NAME, body=query)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['material_text'])

# Realizando indexação Semantica

In [ ]:
from sentence_transformers import SentenceTransformer
modelL6V2 = SentenceTransformer('all-MiniLM-L6-v2')
modelBV2 = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

modelMult = SentenceTransformer('sentence-transformers/quora-distilbert-multilingual')
modelLASE = SentenceTransformer('sentence-transformers/LaBSE')

In [ ]:
len(df_index_catmat)

In [ ]:
df_index_catmat['DESCRICAO_CLASSE'].value_counts()

In [ ]:
# embeddings = modelL6V2.encode(df_materiais['MATERIAL_TEXT'].tolist(), show_progress_bar=True)

In [ ]:
# embeddings = modelL6V2.encode(df_index_catmat['MATERIAL_TEXT'].tolist(), show_progress_bar=True)
embeddingsL6V2 = modelL6V2.encode(df_index_catmat['MATERIAL_TEXT'].tolist(), show_progress_bar=False)
print('modelL6V2 terminado')
embeddingsBV2 = modelBV2.encode(df_index_catmat['MATERIAL_TEXT'].tolist(), show_progress_bar=False)
print('modelBV2 terminado')
embeddingsMult = modelMult.encode(df_index_catmat['MATERIAL_TEXT'].tolist(), show_progress_bar=False)
print('modelMult terminado')
embeddingsLASE = modelLASE.encode(df_index_catmat['MATERIAL_TEXT'].tolist(), show_progress_bar=False)
print('modelLASE terminado')

In [ ]:
embeddingsL6V2[0]
embeddingsBV2[0]
embeddingsMult[0]
embeddingsLASE[244337]


In [ ]:
def doc_generator_sm(index_name, df):
  for idx, row in df.iterrows():
    yield {
        "_index": index_name,
        "_id": idx,
        "_source": {
            "id_material": row['ID_MATERIAL'],
            "codigo_classe": row['CODIGO_CLASSE'],
            "descricao_classe": row['DESCRICAO_CLASSE'],
            "codigo_grupo": row['CODIGO_GRUPO'],
            "caracteristica_busca": row['CARACTERISTICAS_BUSCA'],
            "nome_material": row['NOME_MATERIAL'],
            "material_text": row['MATERIAL_TEXT'],
            "all_mini_base_vector": embeddingsL6V2[idx],
            "bv2_base_vector": embeddingsBV2[idx],
            "mpnet_base_vector": embeddingsMult[idx],
            "labase_base_vector": embeddingsLASE[idx]
        }
    }

In [ ]:
SMNT = "mat_semaanc"

In [ ]:
ctm_doc = doc_generator_sm(SMNT, df_index_catmat)

In [ ]:
es.options(ignore_status=[400,404]).indices.delete(index=SMNT)

body = {
    "settings": {
        'number_of_shards': 3,
        'number_of_replicas': 1,
        "analysis": {
            "analyzer": {
                "standard_asciifolding": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [ "lowercase", "asciifolding" ]
                }
            }
        }
    },
    
    'mappings': {
        'properties': {
            'id_material': {'type': 'long' },
            'codigo_classe': {'type': 'keyword' },
            'descricao_classe': {'type': 'text' },
            'codigo_grupo': {'type': 'keyword' },
            'caracteristica_busca': {'type': 'text' },
            'nome_material': {'type': 'text' },
            'material_text': {'type': 'text' },
            'all_mini_base_vector': {
                'type': 'dense_vector',
                'dims': 384,
                'index': True,
                'similarity': 'cosine'
            },
            'bv2_base_vector': {
                'type': 'dense_vector',
                'dims': 768,
                'index': True,
                'similarity': 'cosine'
            },
            'mpnet_base_vector': {
                'type': 'dense_vector',
                'dims': 768,
                'index': True,
                'similarity': 'cosine'
            },
            'labase_base_vector': {
                'type': 'dense_vector',
                'dims': 768,
                'index': True,
                'similarity': 'cosine'
            }
        }
    }

}

es.indices.create(index=SMNT, body=body)

In [ ]:
indices = es.cat.indices(h='index', format='json')
nomes_indices = [indice['index'] for indice in indices]
nomes_indices

In [ ]:
tamanho_lote = 100
dcmen = list(ctm_doc)
num_batches = (len(dcmen) + tamanho_lote - 1) // tamanho_lote 

batches = [dcmen[i * tamanho_lote:(i + 1) * tamanho_lote] for i in range(num_batches - 1)] 
if len(dcmen) % tamanho_lote <= tamanho_lote:
    ultimo_batch = dcmen[(num_batches - 1) * tamanho_lote:]
    batches.append(ultimo_batch)



In [ ]:
import traceback

for batch in batches:
    
    try:
        response = helpers.bulk(es, batch)

        if response[1]:
            errors = response[1]
            for error in errors:
                print("Falha de indexação:", error)
    except helpers.BulkIndexError as e:
        for err in e.errors:
            print("Falha de indexação:", err)
        traceback.print_exc()


In [ ]:
query_embedding = modelBV2.encode("Bandeja Revelação Material Plástico Características Adicionais Ranhuras Fundo E Apoio Para Termômetro Altura 7L/MIN Comprimento 18POL Largura 13 Cor Bege")

query_smt= {
    'field': 'bv2_base_vector',
    'query_vector': query_embedding.tolist(),
    'k': 10,
    'num_candidates': 10
}


results = es.search(index=SMNT, knn=query_smt, source=['descricao_produto'])

results = [
    {
        "_score": hit["_score"],
        "_source": hit["_source"]
    }
    for hit in results["hits"]["hits"]
]

for hit in results:
    print("Score:", hit["_score"])
    print("Source:", hit["_source"])


# Realizando mapeamento de relevância

In [ ]:
dv_relev_novo = dv_relev_sample.reset_index(drop =True)
dv_relev_novo['CODIGO_BARRA']


In [ ]:
ruidos = ['query', 'sinonimos', 'erros_digitacao', 'reordenados']


In [ ]:
sample_catmat = pd.read_csv('sample_catmat_test.csv')
sample_catmat

In [ ]:
sample_catmat = pd.read_csv('resample_test.csv')
sample_catmat['relevantes'] = None
for idx, query in sample_catmat.iterrows():
    nome_mater = query['NOME_MATERIAL']
    results_descricao = df_index_catmat[df_index_catmat['NOME_MATERIAL'] == nome_mater]
    sample_catmat['relevantes'][idx] = [(row['ID_MATERIAL'], 2) for _, row in results_descricao.iterrows()]
sample_catmat = sample_catmat[['ID_MATERIAL', 'NOME_MATERIAL', 'MATERIAL_TEXT', 'relevantes']]
sample_catmat
    

In [ ]:
queries = []
for idx, row in sample_catmat.iterrows():
    queries.append({'idx':idx,'query': row['MATERIAL_TEXT'], 'page_size': 100})

result_queries = []
for q in queries:
     q_elastic = copy.deepcopy(q)
     q_elastic['abordagem'] = 'lexico'
     result_queries.append(q_elastic)
     q_sql = copy.deepcopy(q)
     q_sql['abordagem'] = 'semantico'
     result_queries.append(q_sql)

df_queries_relv = pd.DataFrame(result_queries)
df_queries_relv

In [ ]:
df_erros_dig = pd.read_csv('erros_digitacao_resample.csv')
# df_sinonimos = pd.read_csv('sinonimos_resample.csv')
df_termos_reord = pd.read_csv('reordenados_resample.csv')


In [ ]:
!export GROQ_API_KEY="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"
from groq import Groq
import os
os.environ['GROQ_API_KEY'] = "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"
api_key = os.environ.get("GROQ_API_KEY")
print(f"GROQ_API_KEY: {api_key}")
import datetime

In [ ]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def generate_synonym_descriptions(df):
    instructions = """
    ### Instruções:
    Abaixo você receberá descrições de produtos. Para cada descrição, gere apenas uma variação que se refere ao mesmo produto, respeitando os valores como altura, profundidade e volume. As unidades de medida podem ser substituídas por sua nomenclatura por extenso (por exemplo, cm para "centímetros").

    ### Exemplos:

    #### Exemplo 1:
    #### Descrição original:
    Forno Mufla Aplicação Tratamento Térmico Temperatura Máxima 1.400V Largura 600MBPS Altura 400V Profundidade 680MMHG Volume 163MM

    #### Variações:
    1. Forno Mufla para Tratamento Térmico com Temperatura Máxima de 1.400 Volts, Largura de 600 Megabits por Segundo, Altura de 400 Volts, Profundidade de 680 Milímetros de Mercúrio e Volume de 163 Milímetros
    2. Forno Mufla para Processos Térmicos, alcançando até 1.400V, com Largura de 600MBPS, Altura de 400V, Profundidade de 680MMHG e Volume de 163MM
    3. Forno Mufla adequado para Tratamento Térmico com Temperatura Máxima de 1.400V, Largura de 600MBPS, Altura de 400V, Profundidade de 680MMHG e Volume de 163MM

    #### Exemplo 2:
    #### Descrição original:
    Geladeira Industrial Capacidade 500L Temperatura mínima -20°C Temperatura máxima 10°C Consumo energético 200W Dimensões 180X70X60CM Peso 80KG

    #### Variações:
    1. Geladeira Industrial com Capacidade de 500 Litros, Temperatura Mínima de -20 Graus Celsius e Máxima de 10 Graus Celsius, Consumo Energético de 200 Watts, Dimensões de 180 por 70 por 60 Centímetros e Peso de 80 Quilogramas
    2. Refrigerador Industrial com Volume de 500L, suportando Temperaturas de -20°C a 10°C, Consumo de Energia de 200W, Medidas de 180X70X60CM e Peso de 80KG
    3. Unidade de Refrigeração Industrial com Capacidade de 500L, operando entre -20°C e 10°C, Consumo de 200W, Dimensões de 180X70X60CM e Massa de 80KG

    ### Agora, gere um prompt de resposta sem nenhuma mensagem de introdução, apenas apresente uma resposta no seguinte formato:
    1.
    Gere apenas uma descrição contendo sinônimos para o seguinte produto:
    """

    data = []

    for index, row in df.iterrows():
        product_description = f"####\n{row['query']}\n\n####:\n"
        completion = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {
                    "role": "user",
                    "content": instructions + product_description
                }
            ],
            temperature=0.9,
            max_tokens=2048,
            top_p=0.95,
            stream=False,
            stop=None,
        )
        
        generated_variation = completion.choices[0].message.content.strip()
        print(generated_variation)

        data.append({
            "query": row['query'],
            "com_sinonimos": generated_variation
        })

        data.append({
            "query": row['query'],
            "com_sinonimos": generated_variation
        })
        
        print(generated_variation)
    df_sinonimo = pd.DataFrame(data)
    
    return df_sinonimo




In [ ]:
df_catmat_llm = pd.read_csv('queries_catmat_ruidos_a.csv')
df_catmat_llm

In [ ]:
df_sinonimos = generate_synonym_descriptions(df_catmat_llm)

In [ ]:
df_sinonimos

In [ ]:
import pandas as pd

def tratar_descricao(df, coluna):
    def remover_prefixo(descricao):
        if descricao.startswith("1. "):
            return descricao[3:]
        return descricao

    df[coluna] = df[coluna].apply(remover_prefixo)
    return df


In [ ]:
df_sinonimos_catm_llam = tratar_descricao(df_sinonimos, 'com_sinonimos')
df_sinonimos_catm_llam

In [ ]:
df_sinonimos_catm_llam.to_csv('sinonimos_llama.csv')

In [ ]:
dv_relev_novo_reord = df_termos_reord.reset_index(drop =True)
dv_relev_novo_erro = df_erros_dig.reset_index(drop =True)
dv_relev_sin = df_sinonimos_catm_llam.reset_index(drop =True)

In [ ]:
dv_relev_sin

In [ ]:
# df_queries_relv['sinonimos'] = dv_relev_sin['com_sinonimos']
df_catmat_llm['sinonimos'] = dv_relev_sin['com_sinonimos']
df_catmat_llm.to_csv('queries_catmat_ruidos_ctm.csv')

In [ ]:
df_queries_relv['erros_digitacao'] = dv_relev_novo_erro['erros_digitacao']
df_queries_relv['reordenados'] = dv_relev_novo_reord['reordenados']
df_queries_relv['sinonimos'] = dv_relev_sin['com_sinonimos']
df_queries_relv

In [ ]:
erros_a = df_queries_relv['query']
erros_a.to_csv('erros_a.csv')

In [ ]:
df_queries_relv.to_csv('queries_catmat_ruidos_ctm.csv')

In [ ]:
df_queries_relv

In [ ]:
df_queries_relv.isnull().sum()

In [ ]:
df_column_query = df_queries_relv['query']
df_column_query

In [ ]:
df_column_query.drop_duplicates(inplace=True)
df_column_query.to_csv('/home/melquicarvalho/Documentos/tcctest/queries_relv_descricoes.csv')

In [ ]:
indices = es.cat.indices(h='index', format='json')
nomes_indices = [indice['index'] for indice in indices]
nomes_indices

In [ ]:
index = "mat_semaanc"
index_info = es.cat.indices(index=index, v=True)
index_info

In [ ]:

from requests.auth import HTTPBasicAuth

logging.basicConfig(filename= 'experiment_relv_catmat.log',
    filemode='a',
    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
    datefmt='%H:%M:%S',
    level=logging.DEBUG)
    
def run(file):
    total_time = 0
    processed_queries = 0
    experiment_queries = pd.read_csv(file)
    for key, value in experiment_queries.iterrows():
        print('key', key)
        # print('value', value)

        logging.info(f'Iniciando processamento da query {value["query"]} com pagina de tamanho {value["page_size"]}')
        try:
            for i in ['query', 'sinonimos', 'erros_digitacao', 'reordenados']:
                match_query = ''
                query = ''        
                if(value['abordagem']) == 'semantico':
                    index_consulta = 'mat_semaanc'
                    match_query = modelL6V2.encode(value[i])
                    
                    query = {
                        "query": {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": "cosineSimilarity(params.query_vector, 'all_mini_base_vector') + 1.0",
                                    "params": {"query_vector": match_query.tolist()}
                                }
                            }
                        },"size": 100
                    }
                else:
                    index_consulta = 'matlex'
                    match_query = value[i]
                    
                    query = {
                        "query": {
                            "match": {
                                "descricao_produto": match_query
                            }
                        },"size": 100
                    }              
        
                sucess = False
                retry_times = 0
                # print('abordagem', value['abordagem'])

                while(not sucess and retry_times < 10):

                    start_time = time.perf_counter()
                    result = requests.get('http://150.165.75.163:9202/' + index_consulta + '/_search', json=query, headers={'Content-type': 'application/json'},
                                          auth=HTTPBasicAuth('elastic', 'teste123'))
                    time_elapsed = time.perf_counter() - start_time
                    total_time += time_elapsed
                    # print('result', result)

                    if result.status_code == requests.codes.ok:
                        sucess = True
                    else:
                        retry_times += 1
                    # print('result', result)
                processed_queries += 1
                result_query = 'result_' + i 
                # print('query', result.content)

                experiment_queries.loc[key, result_query] = result.content.decode('utf-8')

                remaining_time = (total_time / processed_queries) * (len(experiment_queries) - processed_queries)

                logging.info(f'A query levou {time_elapsed:.6f}s')
                logging.info(f'Faltam aproximadamente {int(remaining_time // 3600)}h, {int((remaining_time % 3600) // 60)}m e {math.floor(remaining_time % 60)}s')
            
        except Exception as error:
            logging.error(f'Erro na consulta: {error}')
    experiment_queries.to_csv('queries_result_catmat_llam.csv')



In [ ]:
run('/mnt/DADOS/melqui/busca-smt/queries_catmat_ruidos_a.csv')


In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy

In [ ]:
results = pd.read_csv('queries_result_catmat_llam.csv')
results

In [ ]:
results[results['sinonimos'].isnull()]


In [ ]:
results.isnull().sum()

In [ ]:
for i in ['query', 'sinonimos', 'erros_digitacao', 'reordenados']:
    result_query = 'result_' + i
    print(result_query)
    results[result_query] = results.apply(lambda row: json.loads(row[result_query]), axis=1)

In [ ]:
for idx, row in results.iterrows():
    result = row['result_query']
    if 'hits' in result:
        print(result['hits']['hits'])


In [ ]:
def precision(results, expected):
    hits = 0
    lista = [x[0] for x in expected]
    for result in results:
        if result['id_material'] in lista:
            hits += 1
    if len(results) == 0:
        return 0
    precision_value = hits / len(results)
    return precision_value


In [ ]:
def recall(results, expected, abordagem, size_q, query_exp, cont):
    expected_ids = set(map(lambda x: x[0], expected))
    returned_ids = set(map(lambda x: x['id_material'], results))
    intersection = expected_ids.intersection(returned_ids)
    recaal = len(intersection) / len(expected) if len(expected) > 0 else 0
    return recaal

In [ ]:
nlp = spacy.load('pt_core_news_sm')
def custom_preprocessor(text):
    doc = nlp(text)
    lemmatized_words = [token.lemma_ for token in doc]
    return ' '.join(lemmatized_words)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=custom_preprocessor, lowercase=True, strip_accents='unicode', ngram_range=(1, 3))
tfidf_vectorizer.fit(df_index_catmat['MATERIAL_TEXT'])

In [ ]:
def is_similar(desc1, desc2):
    vectors = tfidf_vectorizer.transform([desc1, desc2])

    vector_emb1 = modelL6V2.encode(desc1) 
    vector_emb1 = vector_emb1.reshape(1, -1)

    vector_emb2 = modelL6V2.encode(desc2)
    vector_emb2 = vector_emb2.reshape(1, -1)
    
    cosine_dist_emb = cosine_similarity(vector_emb1, vector_emb2)
    cosine_dist_tf_idf = cosine_similarity(vectors[0], vectors[1])
    
    value_cosi =  max(cosine_dist_tf_idf, cosine_dist_emb)
    if value_cosi > 0.7:
        return True
    else:
        return False

In [ ]:
def ndcg(results, expected, query):
    annoted_results = []
    for result in results:
        r_copy = copy.deepcopy(result)
        filtered = list(filter(lambda x: x[0] == result['id_material'], expected))
        r_copy['relevance'] = filtered[0][1] if len(filtered) > 0 else 1 if is_similar(result['material_text'], query) else 0
        annoted_results.append(r_copy)
    
    dcg = 0
    idcg = 0
    sorted_results = sorted(annoted_results, key=lambda x: x['relevance'], reverse=True)
    for i in range(len(results)):
        dcg += (2**annoted_results[i]['relevance'] - 1) / math.log2(i + 2)
        idcg += (2**sorted_results[i]['relevance'] - 1) / math.log2(i + 2)
    if idcg == 0: return 0
    return dcg / idcg

In [ ]:
results

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
df_geral = []

In [ ]:
import ast
ruidos = ['query', 'sinonimos', 'erros_digitacao', 'reordenados']
queries_df = pd.read_csv('sample_catmat_test.csv')
queries_df['relevantes'] = queries_df.apply(lambda row: ast.literal_eval(row['relevantes']), axis=1)

cont= 0
processed = 0
size = len(results)
# size_queries = [1,2,3,5,10,20,25,30,40,50]
size_queries = [30]

for size_q in size_queries:
    results_metricas = pd.DataFrame()
    results_metricas['abordagem'] = None
    results_metricas['size_page'] = None
    for i in ruidos:
        results_metricas['precision_' + i] = None
        results_metricas['recall_' + i] = None
        results_metricas['f1-score_' + i] = None
        results_metricas['ndcg_' + i] = None
    for idx, row in results.iterrows():
        results_metricas.loc[idx, 'size_page'] = size_q
                
        processed += 1
        abord = results.loc[idx, 'abordagem']
        print('abord', abord)
        expected = queries_df.loc[row['idx'], 'relevantes']
        abordagem = results.loc[idx, 'abordagem']
        results_metricas.loc[idx, 'abordagem'] = abordagem
        if processed % 500 == 0:
            print(f'{processed} de {size} ({(processed/size)*100:.2f}%)')
        for i in ruidos:
            result = row['result_' + i]
            items = result['hits']['hits']
            query_expected = queries_df.loc[row['idx'], 'MATERIAL_TEXT']
            print('query', query_expected)
            if items is not None:
                conteudos = [objeto['_source'] for objeto in items]
                precision_column = 'precision_' + i
                recall_column = 'recall_' + i
                print('conteudo', conteudos)
                print('expected', expected)
                conteudosSlice = conteudos[:size_q]

                results_metricas.loc[idx, 'precision_' + i] = precision(conteudosSlice, expected)
                results_metricas.loc[idx, 'recall_' + i] = recall(conteudosSlice, expected, abordagem, size_q, query_expected, cont)
                results_metricas.loc[idx, 'f1-score_' + i] = (2 * results_metricas.loc[idx, precision_column] * results_metricas.loc[idx, recall_column]) / (results_metricas.loc[idx, precision_column] + results_metricas.loc[idx, recall_column]) if results_metricas.loc[idx, precision_column] + results_metricas.loc[idx, recall_column] > 0 else 0
                results_metricas.loc[idx, 'ndcg_' + i] = ndcg(conteudosSlice, expected, query_expected)
    df_geral.append(results_metricas)
df_por_sizes = pd.concat(df_geral)

# Métricas por tamanho da page

In [ ]:
df_por_sizes = pd.read_csv('df_por_sizes.csv')
df_por_sizes

In [ ]:
df_por_sizes.to_csv('df_por_sizes_catm.csv')
df_por_sizes

# is similar


In [ ]:
grouped = df_por_sizes.groupby(['size_page', 'abordagem'], as_index=False).agg({'ndcg_query': 'mean', 'precision_query': 'mean', 'f1-score_query': 'mean', 'recall_query': 'mean'})
grouped

In [ ]:
grouped = df_por_sizes.groupby(['size_page', 'abordagem'], as_index=False).agg({'ndcg_erros_digitacao': 'mean', 'precision_erros_digitacao': 'mean', 'f1-score_erros_digitacao': 'mean', 'recall_erros_digitacao': 'mean'})
grouped

In [ ]:
grouped = df_por_sizes.groupby(['size_page', 'abordagem'], as_index=False).agg({'ndcg_reordenados': 'mean', 'precision_reordenados': 'mean', 'f1-score_reordenados': 'mean', 'recall_reordenados': 'mean'})
grouped

In [ ]:
grouped = df_por_sizes.groupby(['size_page', 'abordagem'], as_index=False).agg({'ndcg_sinonimos': 'mean', 'precision_sinonimos': 'mean', 'f1-score_sinonimos': 'mean', 'recall_sinonimos': 'mean'})
grouped

In [ ]:
results_metricas.groupby(['abordagem']).agg(
    {'ndcg_reordenados': ['mean', 'std'], 'precision_reordenados': ['mean', 'std'], 'f1-score_reordenados': ['mean', 'std'], 'recall_reordenados': ['mean', 'std']})

In [ ]:
processed = 0
size = len(results)
teste_2 = pd.DataFrame()
for idx, row in results.iterrows():
    for i in ruidos:
        results_ruidos = row['result_' + i]
        items = results_ruidos['hits']['hits']
        conteudos = [objeto['_source'] for objeto in items]
        processed += 1
        if processed % 100 == 0:
            print(f'{processed} de {size} ({(processed/size)*100:.2f}%)')
        
        query_expected = queries_df.loc[row['idx'], 'DESCRICAO_PRODUTO']
        expected = queries_df.loc[row['idx'], 'relevantes']
        annoted_results = []
        for result in conteudos:
            r_copy = copy.deepcopy(result)
            filtered = list(filter(lambda x: x[0] == result['id'], expected))
            r_copy['relevance'] = filtered[0][1] if len(filtered) > 0 else 1 if is_similar(result['descricao_produto'], query_expected) else 0
            annoted_results.append(r_copy)
        teste_2['result_' + i] = annoted_results
teste_2    

In [ ]:
results_100 = results[results['page_size'] == 100]

processed = 0
size = len(results_100)
for idx, row in results_100.iterrows():
    processed += 1
    if processed % 100 == 0:
        print(f'{processed} de {size} ({(processed/size)*100:.2f}%)')
    items = result['hits']['hits']
    conteudos = [objeto['_source'] for objeto in items]
            
    expected = queries_df.loc[row['idx'], 'relevantes']
    annoted_results = []
    for result in conteudos:
        r_copy = copy.deepcopy(result)
        filtered = list(filter(lambda x: x[0] == result['id'], expected))
        r_copy['relevance'] = filtered[0][1] if len(filtered) > 0 else 1 if is_similar(result['descricao'], query) else 0
        annoted_results.append(r_copy)
    row['result'] = annoted_results

In [ ]:
df_por_sizes = pd.read_csv('df_por_sizes.csv')
df_por_sizes

In [ ]:
grouped = df_por_sizes.groupby(['size_page', 'abordagem'], as_index=False).agg({
    'ndcg_erros_digitacao': 'mean', 'precision_erros_digitacao': 'mean', 'f1-score_erros_digitacao': 'mean', 'recall_erros_digitacao': 'mean',
    'ndcg_query': 'mean', 'precision_query': 'mean', 'f1-score_query': 'mean', 'recall_query': 'mean',
    'ndcg_sinonimos': 'mean', 'precision_sinonimos': 'mean', 'f1-score_sinonimos': 'mean', 'recall_sinonimos': 'mean',
    'ndcg_reordenados': 'mean', 'precision_reordenados': 'mean', 'f1-score_reordenados': 'mean', 'recall_reordenados': 'mean'
    })
grouped

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

lexico = grouped[grouped['abordagem'] == 'lexico']   

semantico = grouped[grouped['abordagem'] == 'semantico']   
_, ax = plt.subplots(2, 2, figsize=(15,10))
ax[0,0].set_title('reordenados')
ax[0,0].set_xlabel('Tamanho da página')
ax[0,0].set_ylabel('Pontuação de NDCG')
ax[0,0].plot(lexico['size_page'], lexico['precision_reordenados'], label='lexico')
ax[0,0].plot(semantico['size_page'], semantico['precision_reordenados'], label='semantico', color='red', linewidth=1.0)
ax[0,0].legend()

ax[0,1].set_title('erros de digitação')
ax[0,1].set_xlabel('Tamanho da página')
ax[0,1].set_ylabel('Pontuação de NDCG')
ax[0,1].plot(lexico['size_page'], lexico['precision_erros_digitacao'], label='lexico')
ax[0,1].plot(semantico['size_page'], semantico['precision_erros_digitacao'], label='semantico', color='red', linewidth=1.0)
ax[0,1].legend()

ax[1,0].set_title('sinonimos')
ax[1,0].set_xlabel('Tamanho da página')
ax[1,0].set_ylabel('Pontuação de NDCG')
ax[1,0].plot(lexico['size_page'], lexico['precision_sinonimos'], label='lexico')
ax[1,0].plot(semantico['size_page'], semantico['precision_sinonimos'], label='semantico', color='red', linewidth=1.0)
ax[1,0].legend()

ax[1,1].set_title('query')
ax[1,1].set_xlabel('Tamanho da página')
ax[1,1].set_ylabel('Pontuação de NDCG')
ax[1,1].plot(lexico['size_page'], lexico['precision_query'], label='lexico')
ax[1,1].plot(semantico['size_page'], semantico['precision_query'], label='semantico', color='red', linewidth=1.0)
ax[1,1].legend()

plt.tight_layout()
plt.gcf().savefig('precision_por_size.png', dpi=300)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
grouped = df_por_sizes.groupby(['size_page', 'abordagem'], as_index=False).agg({'ndcg_query': 'mean', 'precision_query': 'mean', 'f1-score_query': 'mean', 'recall_query': 'mean'})
grouped
for name, group in grouped:
    plt.plot(group['precision_query'], group['size_page'], marker='o', label=group['abordagem'])

plt.title('Relação entre Pontuação e Tamanho por Tipo de Teste')
plt.xlabel('Pontuação')
plt.ylabel('Tamanho')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
elastic_data = grouped[grouped['sgbd'] == 'elastic']
elastic_semantic_data = grouped[grouped['sgbd'] == 'elastic_semantic']
sql_server_data = grouped[grouped['sgbd'] == 'sql server']

metrics = ['ndcg', 'precision', 'recall', 'f1-score']
metric_titles = ['NDCG', 'Precision', 'Recall', 'F1-score']
xa = ['a)', 'b)', 'c)', 'd)']

plt.style.use('custom_charts.mplstyle')
fig = plt.figure(figsize=(28, 16))

for i, metric in enumerate(metrics):
    ax = fig.add_subplot(2, 2, i+1)
    x = range(len(elastic_data))

    ax.plot(range(len(elastic_data)), elastic_data[metric].tolist(), '-o', label=f'Elasticsearch {metric}')
    ax.plot(range(len(elastic_data)), elastic_semantic_data[metric].tolist(), '-o', label=f'Elasticsearch Semantic {metric}')
    ax.plot(range(len(elastic_data)), sql_server_data[metric].tolist(), '-x', label=f'SQL Server {metric}')
    ax.set_ylabel(xa[i] + ' ' + metric_titles[i])
    ax.set_xticks(x)
    ax.set_xlabel('k')
    ax.legend()
    ax.set_xticklabels(elastic_data['page_size'])

plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 10))
plt.plot(relevantes_by_p[relevantes_by_p['sgbd'] == 'elastic']['position'], relevantes_by_p[relevantes_by_p['sgbd'] == 'elastic']['prop'], label='Elasticsearch', color='blue')
plt.plot(relevantes_by_p[relevantes_by_p['sgbd'] == 'elastic semantic']['position'], relevantes_by_p[relevantes_by_p['sgbd'] == 'elastic semantic']['prop'], label='Elasticsearch Semantic', color='red')
plt.plot(relevantes_by_p[relevantes_by_p['sgbd'] == 'sql server']['position'], relevantes_by_p[relevantes_by_p['sgbd'] == 'sql server']['prop'], label='Sql Server', color='orange')
plt.plot(relevantes_by_p[relevantes_by_p['sgbd'] == 'sql server']['position'], relevantes_by_p[relevantes_by_p['sgbd'] == 'sql server']['prop_gabarito'], label='Ideal', color='green')
plt.xlabel('Posição nos resultados')
plt.ylabel('Porcentagem de relevântes')
plt.legend()

In [ ]:
grouped = df_por_sizes.groupby(['size_page', 'abordagem'], as_index=False).agg({'ndcg_sinonimos': 'mean', 'precision_sinonimos': 'mean', 'f1-score_sinonimos': 'mean', 'recall_sinonimos': 'mean'})
grouped